## Tutorial 2: Chords

In [18]:
from gsapi import *

The GS-API provides a detailed logging system to help debugging. You can set different logging levels for the different modules in the library:

In [19]:
gsio.gsioLog.setLevel(level=logging.INFO)

Now, let's use "gsio.fromMidiFile" to load a regular MIDI file onto a Pattern p:

In [20]:
p = gsio.fromMidiFile("../corpora/harmony/I5-IV.mid", "pitchName")
print(p)

[gsapi.gsio     ][INFO]  start processing I5-IV.mid (gsio.py:136)
[gsapi.gsio     ][INFO]  I5-IV.mid: getting track: 0 Yamaha Grand Piano (gsio.py:161)
[gsapi.gsio     ][INFO]  set duration 3.950000 at start 0.050000  (gsio.py:208)
[gsapi.gsio     ][INFO]  set duration 3.950000 at start 0.050000  (gsio.py:208)
[gsapi.gsio     ][INFO]  set duration 3.950000 at start 0.050000  (gsio.py:208)
[gsapi.gsio     ][INFO]  set duration 2.250000 at start 4.000000  (gsio.py:208)
[gsapi.gsio     ][INFO]  set duration 2.250000 at start 4.000000  (gsio.py:208)
[gsapi.gsio     ][INFO]  set duration 2.250000 at start 4.000000  (gsio.py:208)
[gsapi.gsio     ][INFO]  set duration 1.250000 at start 6.500000  (gsio.py:208)
[gsapi.gsio     ][INFO]  set duration 1.250000 at start 6.500000  (gsio.py:208)
[gsapi.gsio     ][INFO]  set duration 1.250000 at start 6.500000  (gsio.py:208)


Name: I5-IV.mid
Duration: 7.75
BPM: 125.00
TimeSignature: 16/16
Key: C
FilePath: None

D3 50 80 0.0500 3.9500
A3 57 80 0.0500 3.9500
D4 62 80 0.0500 3.9500
D3 50 80 4.0000 2.2500
G3 55 80 4.0000 2.2500
B3 59 80 4.0000 2.2500
D3 50 80 6.5000 1.2500
G3 55 80 6.5000 1.2500
B3 59 80 6.5000 1.2500



In this example, we are going to take a midi file and transpose its content so that its first note is C4.
For that, we can request the start time of the first event in the pattern:

In [6]:
first_event_time = p.events[0].startTime
print(first_event_time)

0.05


And get all the vertical components at start time, in case there is a chord:

In [7]:
first_chord = p.getStartingEventsAtTime(first_event_time)
print(first_chord)

[D3 50 80 0.0500 3.9500, A3 57 80 0.0500 3.9500, D4 62 80 0.0500 3.9500]


We filter the midi note numbers of the first chord:

In [8]:
first_notes_midi = []
for e in first_chord:
    first_notes_midi.append(e.pitch)
print(first_notes_midi)

[50, 57, 62]


And make sure they are sorted in ascending order. Then we take the lowest note of the aggregate:

In [9]:
first_notes_midi.sort()
tonic = first_notes_midi[0]
print(tonic)

50


Now we can find a transposition factor and transpose the pattern so that the first note (or lowest note of the first chord) is C4:

In [10]:
transposition_interval = 60 - tonic
p.transpose(transposition_interval)
print(p)

Name: I5-IV.mid
Duration: 7.75
BPM: 125.00
TimeSignature: 16/16
Key: C
FilePath: None

['C4'] 60 80 0.0500 3.9500
['G4'] 67 80 0.0500 3.9500
['C5'] 72 80 0.0500 3.9500
['C4'] 60 80 4.0000 2.2500
['F4'] 65 80 4.0000 2.2500
['A4'] 69 80 4.0000 2.2500
['C4'] 60 80 6.5000 1.2500
['F4'] 65 80 6.5000 1.2500
['A4'] 69 80 6.5000 1.2500



There are a few methods available to perform sanity check of the GS Pattern and reformat it if necessary. The following methods force that events in the GS Pattern are ordered strictly in chronological order after manipulations and without overlapping notes:


In [11]:
p.reorderEvents()
p.removeOverlapped(usePitchValues=True)

Normally we would quantize the pattern, so its durations and onsets align to the desired resolution.

In [12]:
p.quantize(0.25, quantizeStartTime=True, quantizeDuration=True)
print(p)

Name: I5-IV.mid
Duration: 7.75
BPM: 125.00
TimeSignature: 16/16
Key: C
FilePath: None

['C4'] 60 80 0.0000 3.7500
['G4'] 67 80 0.0000 3.7500
['C5'] 72 80 0.0000 3.7500
['C4'] 60 80 4.0000 2.2500
['F4'] 65 80 4.0000 2.2500
['A4'] 69 80 4.0000 2.2500
['C4'] 60 80 6.5000 1.2500
['F4'] 65 80 6.5000 1.2500
['A4'] 69 80 6.5000 1.2500



After quantisation, you could create "silence" events filling empty time intervals (if any) in order to export the pattern to a score software.

In [13]:
p.fillWithSilences()
print(p)

Name: I5-IV.mid
Duration: 7.75
BPM: 125.00
TimeSignature: 16/16
Key: C
FilePath: None

['C4'] 60 80 0.0000 3.7500
['G4'] 67 80 0.0000 3.7500
['C5'] 72 80 0.0000 3.7500
silence 0 0 3.7500 0.2500
['C4'] 60 80 4.0000 2.2500
['F4'] 65 80 4.0000 2.2500
['A4'] 69 80 4.0000 2.2500
silence 0 0 6.2500 0.2500
['C4'] 60 80 6.5000 1.2500
['F4'] 65 80 6.5000 1.2500
['A4'] 69 80 6.5000 1.2500



Alternatively, you can fill the gaps (silences) with the duration of the previous sounding event(s)

In [14]:
p.removeByTags(["silence"])
p.fillWithPreviousEvent()
p.fillWithSilences()
print(p)

Name: I5-IV.mid
Duration: 7.75
BPM: 125.00
TimeSignature: 16/16
Key: C
FilePath: None

['C4'] 60 80 0.0000 4.0000
['G4'] 67 80 0.0000 4.0000
['C5'] 72 80 0.0000 4.0000
['C4'] 60 80 4.0000 2.5000
['F4'] 65 80 4.0000 2.5000
['A4'] 69 80 4.0000 2.5000
['C4'] 60 80 6.5000 1.2500
['F4'] 65 80 6.5000 1.2500
['A4'] 69 80 6.5000 1.2500



As we did before, we can export all the modifications onto a new MIDI file:

In [16]:
gsio.toMidiFile(p, folderPath='../../output', name='transposed')

480
(0, 0)
0.0 480
0.0 480
0.0 480
4.0 480
4.0 480
4.0 480
6.5 480
6.5 480
6.5 480


'../../output/transposed.mid'

In [ ]:
# chord = Chordify(myPattern) 

#print(chord.outputPattern)
#print(type(chord.outputPattern))


# chord_name = Chord()
# chord_name.getDescriptorForPattern(chord.outputPattern[0])

# io.toMidi(myPattern, path='./', name='tests')

# needs notepad or musescore installed
#s = converter.parse('./test.mid')
#print s
#s.show()